In [4]:
from pymongo import MongoClient
from datetime import datetime

# Establishing connection
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

#schema validation. refers to 6-1
user_validator = {
    "$jsonSchema": {
        "bsonType": "object",
        "required": ["userId", "email", "firstName", "lastName", "role", "dateJoined", "isActive"],
        "properties": {
            "email": {"bsonType": "string", "pattern": "^.+@.+\\..+$"},
            "role": {"enum": ["student", "instructor"]},
            "dateJoined": {"bsonType": "date"},
            "isActive": {"bsonType": "bool"}
        }
    }
}

# --- CREATE COLLECTION ---
# Try/except block runs if the collection already exists
try:
    db.create_collection('users', validator=user_validator)
    print("Collection 'users' created with validation.")
except Exception as e:
    # If the collection exists, MongoDB often raises an error on re-creation.
    print(f"Collection 'users' already exists or validation error: {e}")

# Create other collections 
for col_name in ['courses', 'enrollments', 'lessons', 'assignments', 'submissions']:
    if col_name not in db.list_collection_names():
        db.create_collection(col_name)

Collection 'users' created with validation.
